In [1]:
import pandas as pd
import datetime as dt

In [2]:
from config import sheet_id, sheet_name

In [3]:
import os
os.getcwd()

'c:\\Users\\jluong\\OneDrive - CSU Office of the Chancellor\\Documents\\Basecamp New\\basecamp_salary_dashboard'

In [4]:
# import sheet_id and sheet_name from a local config file
sheet_id = sheet_id
sheet_name = sheet_name
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
# replace whitespace in the URL
url = url.replace(" ", "%20")

In [5]:
# specify columns to read in (all but the "Company Name" column)
df_raw = pd.read_csv(url, usecols = lambda x: x != "Company Name", parse_dates = ['Submitted At'])

In [6]:
df_raw.head()

,I am a:,Industry,Job Title,Location,Flexibility,# Years,Pay,I charge per:,Bonuses/ Commissions,Equity,...,Specialty expertise,Field of work,City,Government role?,Parental leave,Pronouns,Race/Ethnicity,I identify as part of the following communities,Submitted At,Token
0,Full-Time Employee,Active Lifestyle,North America Marketing Manager,Oregon,Hybrid/Work From Home,15+,95000,/Year,10000.0,False,...,NaN,Marketing & Branding,"Portland, OR",Nope,6 months,She/Her,NaN,"LGBTQIA2S+, Woman",2022-10-29 22:59:19,4uhvwonebzd2xry77gpme4uhvwn6pmnd
1,Full-Time Employee,Running,PR Director,Colorado,Fully Remote,8,165000,/Year,NaN,True,...,NaN,PR & Communications,Denver,Nope,3 months,He/Him,Latino/Hispanic,LGBTQIA2S+,2022-10-29 22:56:19,9c19jk3coesvcdu9c19px0ltwrzj1cbn
2,Full-Time Employee,Travel/Tourism,Vice President of Brand Marketing,California,Fully Remote,5,150000,/Year,4000.0,False,...,"Visual / Graphic Design, Project Management, M...",Hospitality & Adventure Tourism,San Francisco,Nope,2 months,She/Her,NaN,Woman,2022-10-29 22:23:42,7bnz7dq3fbcyg3xf0vzx97bnz7pn3bij
3,Full-Time Employee,Bike,Store Manager,Pennsylvania,In-Office,6,66000,/Year,15000.0,True,...,"Retail management, outdoor leadership,",Retail,Philadelphia,Nope,2 months,He/Him,NaN,NaN,2022-10-29 22:14:55,mzpxp2psypsbtt1hcespgmzpxp2p7arl
4,Full-Time Employee,Bike,People Ops,Oregon,Fully Remote,4,80000,/Year,0.0,False,...,Master's Degree; Two certifications to facilit...,Human Resources (seems like it should be an op...,"Bend, OR",Nope,1 months,She/Her,NaN,"50+, LGBTQIA2S+, Woman",2022-10-29 21:58:27,8xv58qt6gkciedqai2v8xv58qt69tmh2


In [7]:
df = df_raw.copy(deep = True)

In [8]:
df.columns

Index(['I am a:', 'Industry', 'Job Title', 'Location', 'Flexibility',
       '# Years', 'Pay', 'I charge per:', 'Bonuses/ Commissions', 'Equity',
       'PTO', 'Training/Education', 'Add'l Qs', 'Benefits', 'OTJ Experience',
       'Starting Pay', 'Specialty expertise', 'Field of work', 'City',
       'Government role?', 'Parental leave ', 'Pronouns', 'Race/Ethnicity',
       'I identify as part of the following communities', 'Submitted At',
       'Token'],
      dtype='object')

In [9]:
# general syntax of loc
# df.loc['row_label', 'column_label']

# If there are multiple labels, they should be specified inside lists:
# df.loc[['row_1', 'row_2'], ['column_1', 'column_2']]


In [10]:
columns_wanted = ['I am a:', 'Industry', 'Location', 'Flexibility', '# Years', 'Pay', 'I charge per:', 'Field of work', 'Submitted At']

In [11]:
df_subset = df.loc[:, columns_wanted]

In [12]:
df_subset.head()

,I am a:,Industry,Location,Flexibility,# Years,Pay,I charge per:,Field of work,Submitted At
0,Full-Time Employee,Active Lifestyle,Oregon,Hybrid/Work From Home,15+,95000,/Year,Marketing & Branding,2022-10-29 22:59:19
1,Full-Time Employee,Running,Colorado,Fully Remote,8,165000,/Year,PR & Communications,2022-10-29 22:56:19
2,Full-Time Employee,Travel/Tourism,California,Fully Remote,5,150000,/Year,Hospitality & Adventure Tourism,2022-10-29 22:23:42
3,Full-Time Employee,Bike,Pennsylvania,In-Office,6,66000,/Year,Retail,2022-10-29 22:14:55
4,Full-Time Employee,Bike,Oregon,Fully Remote,4,80000,/Year,Human Resources (seems like it should be an op...,2022-10-29 21:58:27


In [13]:
df_subset.dtypes

I am a:                  object
Industry                 object
Location                 object
Flexibility              object
# Years                  object
Pay                       int64
I charge per:            object
Field of work            object
Submitted At     datetime64[ns]
dtype: object

In [14]:
# Extract the Year from the Submitted At column
df_subset['Year'] = df_subset['Submitted At'].dt.strftime('%Y')

In [15]:
# Rename columns
df_subset.rename(columns = {"# Years": "Years of Experience"}, inplace = True)

In [16]:
df_subset.to_csv("salary_data.csv")

In [17]:
df_subset.Pay.describe()

count       920.000000
mean      57712.026087
std       44773.733138
min           0.000000
25%       30000.000000
50%       58000.000000
75%       80000.000000
max      403950.000000
Name: Pay, dtype: float64

In [18]:
df_subset.Year

0      2022
1      2022
2      2022
3      2022
4      2022
       ... 
915    2023
916    2023
917    2023
918    2023
919    2023
Name: Year, Length: 920, dtype: object